In [1]:
import numpy as np
import os
import glob
import pandas as pd
import tqdm

In [2]:
caption_files = glob.glob("data/captions/*.csv")

In [3]:
x = np.array([]).reshape(0, 1561)
y = np.array([]).reshape(0, 84)

In [4]:
for caption_file in tqdm.tqdm(caption_files):
    
    video_id = caption_file.split("/")[-1].replace(".csv", "")
    text_feature_file = f"data/features/text/{video_id}.npy"

    captions = pd.read_csv(caption_file)

    text_features = np.load(text_feature_file)

    for i in range(len(captions)):
        t, d = captions.iloc[i, [0, 1]]

        audio_feature_file = f"data/features/audio/{video_id}/audio_feat_{t}_{d}.npy"
        hand_pose_file = f"data/features/handpose/{video_id}/handpose_{t}_{d}.npy"

        if os.path.exists(audio_feature_file):
            audio_features = np.load(audio_feature_file)
            
            if os.path.exists(hand_pose_file) and os.path.getsize(hand_pose_file) > 278:
                hand_poses = np.load(hand_pose_file, allow_pickle=True)
                
                x1 = audio_features.reshape(audio_features.shape[0], -1)
                x2 = text_features[i].reshape(1, -1).repeat(audio_features.shape[0], axis=0)
                
                x = np.concatenate((x, np.append(x1, x2, axis=1)), axis=0)
                y = np.concatenate((y, hand_poses), axis=0)


100%|██████████| 58/58 [06:16<00:00,  6.49s/it]


In [5]:
np.savez_compressed("data/dataset.npz", x=x, y=y)

In [6]:
x.shape, y.shape

((99709, 1561), (99709, 84))